# perf improvement for IIR filter

In [3]:
import math
import numpy as np

In [72]:
freq = np.logspace(1+math.log10(2), 4+math.log10(2), 200)

In [73]:
import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.filter_iir import Biquad

In [80]:
peak = Biquad(3, 1000, 48000, 1, 3)

peq_slow = np.array([20.0*math.log10(peak.resultSlow(f)) for f in freq])
peq_fast = np.array([20.0*math.log10(peak.result(f)) for f in freq])

def vresult(biquad, f):
    phi = np.sin(math.pi * f * 2 / (2 * biquad.srate))**2
    phi2 = phi**2
    r = (biquad.r_up0 + biquad.r_up1 * phi + biquad.r_up2 * phi2) / (biquad.r_dw0 + biquad.r_dw1 * phi + biquad.r_dw2 * phi2)
    return np.where(r<=0.0, -200, 20*np.log10(np.sqrt(r)))

peq_vec1 = vresult(peak, freq)
print('Error: {} {}'.format(
    np.max(peq_slow-peq_fast),
    np.max(peq_slow-peq_vec1)))

Error: 1.9984014443252818e-15 1.9984014443252818e-15


In [81]:
%timeit [20.0*math.log10(peak.result(f)) for f in freq]

181 µs ± 547 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [76]:
%timeit [20.0*math.log10(peak.resultSlow(f)) for f in freq]

271 µs ± 810 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [78]:
def vresult(biquad, f):
    phi = np.sin(math.pi * f * 2 / (2 * biquad.srate))**2
    phi2 = phi**2
    r = (biquad.r_up0 + biquad.r_up1 * phi + biquad.r_up2 * phi2) / (biquad.r_dw0 + biquad.r_dw1 * phi + biquad.r_dw2 * phi2)
    return np.where(r<=0.0, -200, 20*np.log10(np.sqrt(r)))

In [79]:
%timeit vresult(peak, freq)

13.9 µs ± 28.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
